# Assignment 2 
### \<Possakorn> \<a1873765>

## A. Tasks as specified for your team structure

**One headings for each task.**

### Read and pre-process

In [69]:
# library
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
import string
import nltk
import re
import spacy
from tqdm.auto import tqdm
import sys
from IPython.display import display, HTML

# Ensure necessary NLTK data is downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

import logging
logging.getLogger().setLevel(logging.WARNING)
logging.getLogger('sentence_transformers').setLevel(logging.WARNING)
logging.getLogger('fastcoref').setLevel(logging.WARNING)

def display_df(df):
	display(HTML(df.to_html()))

[nltk_data] Downloading package punkt to /Users/tytatee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tytatee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/tytatee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [70]:
# identify basic information
print("-----Show the sys info-----", sys.version)

# read csv
df = pd.read_csv('news_dataset.csv', encoding='ISO-8859-1')

# identify basic information
print("-----Show the data summary-----")
print(df.info())

# we will select only 3 topics with strata amount
print("-----Show the topic distribution-----")
df[['topic','article']].groupby('topic').count()

# we show 10 dataset views
print("-----Show 5 dataset views-----")
df.head(5)

-----Show the sys info----- 3.10.13 (main, Aug 24 2023, 12:59:26) [Clang 15.0.0 (clang-1500.1.0.2.5)]
-----Show the data summary-----
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       1000 non-null   int64 
 1   author   994 non-null    object
 2   date     1000 non-null   object
 3   year     1000 non-null   object
 4   month    1000 non-null   object
 5   topic    1000 non-null   object
 6   article  1000 non-null   object
dtypes: int64(1), object(6)
memory usage: 54.8+ KB
None
-----Show the topic distribution-----
-----Show 5 dataset views-----


,id,author,date,year,month,topic,article
0,17307,Marlise Simons,1/01/2017,2017,1,architecture,PARIS ? When the Islamic State was about to...
1,17292,Andy Newman,31/12/2016,2016,12,art,Angels are everywhere in the Mu?iz family?s ap...
2,17298,Emma G. Fitzsimmons,2/01/2017,2017,1,business,Finally. The Second Avenue subway opened in Ne...
3,17311,Carl Hulse,3/01/2017,2017,1,business,WASHINGTON ? It?s or time for Republica...
4,17339,Jim Rutenberg,5/01/2017,2017,1,business,"For Megyn Kelly, the shift from Fox News to NB..."


In [71]:
test_sentence = "Megyn Kelly?s new office at NBC News sits a block north of Fox News headquarters in Midtown Manhattan. But it might as well be a world away. In switching networks at a pivotal point in her career, Ms. Kelly, the No.   personality in cable television news, is taking a calculated risk that she can swap her dedicated Fox audience for the broader, but more fickle, viewership of network television. There are challenges from the  . Her splashy arrival has the potential to fray nerves among the big personalities at the network  ?   who already compete against one another for interviews and scoops. Ms. Kelly, 46, will also be taking on a daytime talk show format that has been a virtual graveyard for television news personalities in the last 10 years. And the new Sunday newsmagazine show that NBC plans to build around Ms. Kelly will go up against a giant that has not been meaningfully challenged for decades: ?60 Minutes? on CBS. Still, Ms. Kelly is a bona fide star with a   book and a breakout role in this year?s presidential campaign, when she clashed with Donald J. Trump. NBC News comes out the winner in one of the most closely watched talent sweepstakes in years, acquiring one of television?s biggest names who could play a role in any number of major network events, like coverage of elections or the Olympics. Interviews on Tuesday with network executives and producers  ?   from Fox, NBC and other rival channels  ?   suggest that Ms. Kelly?s performance at NBC will be as closely watched in the industry as her past few months of contract negotiations. Ms. Kelly will have to design her daytime talk show from scratch. Even though she made her name as a news anchor, she has argued that she is not obsessed with politics. When she hosted a   special on Fox in May  ?   her first major foray outside cable news  ?   she expressed a desire to combine the qualities of Oprah Winfrey, Barbara Walters and Charlie Rose. That special  ?   which featured interviews with Mr. Trump, the celebrity lawyer Robert Shapiro and the actress Laverne Cox  ?   received middling reviews. It was far from a ratings hit: Among adults younger than 50, the demographic most important to broadcasters, Ms. Kelly?s special performed about as well as ABC?s ?Beyond the Tank,? a   spinoff. A daily daytime talk show also poses risks.   talent like Jane Pauley, Meredith Vieira, Katie Couric and Anderson Cooper have taken a stab at the genre in the past, and each one failed. In Ms. Pauley?s case, NBC invested millions of dollars, but the show was yanked in 2005 after just one year. NBC said on Tuesday that Ms. Kelly?s show was expected to be closer to a news program than the typical daytime talk show, although it is unclear what exactly that will mean or how much appetite there is for news amid a landscape including shows like ?Days of Our Lives,? ?The Ellen DeGeneres Show? and ?Steve Harvey. ? The audience for daytime television is also significantly more diverse than Fox?s   viewership. Starting a Sunday rival to ?60 Minutes,? the   among newsmagazines, is likewise no easy task. ?Rock Center,? which Brian Williams hosted, lasted two seasons. NBC?s most recent newsmagazine, ?On Assignment,? ran    against ?60 Minutes? over five weeks the show averaged about four million viewers, compared with more than nine million for ?60 Minutes,? which broadcast two repeats during that time. (This season, ?60 Minutes? is averaging more than 14 million viewers.) It is also unclear how NBC will accommodate Ms. Kelly?s show during the National Football League season, when NBC?s popular ?Sunday Night Football? package includes a highly rated pregame show that begins at 7 p. m. Eastern. But Andrew Lack, NBC News?s chairman, has long had newsmagazines in his blood. In addition to overseeing a ?60 Minutes? competitor on CBS in the 1980s, Mr. Lack presided over NBC in the 1990s when newsmagazines, including the network?s popular ?Dateline,? dominated   lineups. Fox, meanwhile, must now set a course without one of its biggest names, as the network continues to recalibrate itself after the ouster of its chairman, Roger Ailes. Ms. Kelly?s exit from Fox News was so abrupt that it was announced on the day that the network had run a   ad in The Wall Street Journal trumpeting the ratings of its   lineup, with Ms. Kelly prominently pictured. Her departure stunned the Fox newsroom, where journalists and executives spent Tuesday afternoon speculating over which anchor might replace Ms. Kelly in the coveted 9 p. m. slot  ?   and wondering if Ms. Kelly would even appear that night. When Greta Van Susteren, another veteran anchor, announced her departure in September, network representatives visited her home to tell her not to bother coming in. In the end, Ms. Kelly was granted a chance to bid farewell to Fox News viewers  ?   her last show is Friday. It is not clear who will replace her. Fox News has never had an     lineup. Potential replacements being floated inside the network on Tuesday include four women who have regularly filled in for Ms. Kelly: Sandra Smith, a host of Fox?s noon show, ?Outnumbered? Trish Regan, a rising star at Fox Business Network Shannon Bream, who covers the Supreme Court and Martha MacCallum, a morning anchor. Kimberly Guilfoyle, a host of ?The Five? who is friendly with Mr. Trump?s circle, and Tucker Carlson, who has put up high ratings since taking over Fox News?s 7 p. m. slot, have also been suggested. Sean Hannity, whose viewership at 10 p. m. increased enormously in 2016 and has spiked since Election Day, could be moved up an hour, but his momentum in his time slot may make Fox executives reluctant to make a switch. One winner in the sweepstakes for Ms. Kelly could be CNN, even though it did not succeed in recruiting her. CNN, while still behind Fox News in total viewers by a wide margin, has occasionally beaten Fox among viewers 25 to 54, the demographic that determines advertising rates. In 2016, CNN finished within 58, 000 viewers of Fox in prime time in the demographic  ?   cutting Fox?s lead in half. Since the election, Fox has regained a sizable lead, but now, without spending a penny, CNN will now have an opportunity to take another run at Fox?s   advantage."

In [72]:
# data = [
#     {
#         "article_id": article_id,
#         "question": question,
#         "gold_sentence": gold_sentence,
#         "gold_standard": gold_standard
#     } 
# ]
data = [
    {
        "article_id": 17340,
        "info": {
            "gold_sentence" : "Megyn Kelly?s new office at NBC News sits a block north of Fox News headquarters in Midtown Manhattan.",
            "QA": [
                {"question": ['Where Megyn Kelly?s new office?'],"gold_standard": ['Midtown Manhattan']},
                {"question": ['Where Megyn Kelly?s new office?'],"gold_standard": ['Midtown Manhattan']}
			]
			}
		} 
	]
data

[{'article_id': 17340,
  'info': {'gold_sentence': 'Megyn Kelly?s new office at NBC News sits a block north of Fox News headquarters in Midtown Manhattan.',
   'QA': [{'question': ['Where Megyn Kelly?s new office?'],
     'gold_standard': ['Midtown Manhattan']},
    {'question': ['Where Megyn Kelly?s new office?'],
     'gold_standard': ['Midtown Manhattan']}]}}]

In [73]:
#read csv
df_qk = pd.read_csv('sampling_data_quick.csv')
df = pd.read_csv('news_dataset.csv', encoding='ISO-8859-1')
df_sam = df[df['topic'].isin(['politics', 'business','entertainment'])].groupby('topic').sample(n=34, random_state=1311)
df_sam = df_sam.reset_index(drop = True)
df_sam.set_index('id', inplace=True)
df_sam.head(5)
df_dev = pd.read_csv('sampling_data_dev.csv')
df_dev = pd.merge(df_dev, df[['id','article','topic']], on='id', how='inner')
df_dev = df_dev.reset_index(drop = True)
df_dev.set_index('id', inplace=True)
df_dev
df = df.reset_index(drop = True)
df.set_index('id', inplace=True)

In [74]:
def preprocess_text(text, c_stop = True, c_lower = True, c_lemma = True):
	
	# 00 setting

	lemmatizer = WordNetLemmatizer()
	stop_words = set(stopwords.words('english'))
	if c_stop == False:
		stop_words = {}
	
	# 01 Lowercasing the text
	
	if c_lower == True:
		text = text.lower()
	
	# Tokenizing while keeping punctuation"

	words = word_tokenize(text)
 
	# 02 Lemmatization and stopping words removal; keeping "."

	if c_lemma == True:
		filtered_words = [lemmatizer.lemmatize(word) for word in words 
					if word not in stop_words 
					and (word not in string.punctuation or word == '.')]
	else:
		filtered_words = [word for word in words 
					if word not in stop_words 
					and (word not in string.punctuation or word == '.')]
	
	# Joining the processed words back into a string, preserving punctuation

	return " ".join(filtered_words)

print("Check the function preprocess")
print(f"Before: {test_sentence}")
print(f"After: {preprocess_text(test_sentence)}")


Check the function preprocess
Before: Megyn Kelly?s new office at NBC News sits a block north of Fox News headquarters in Midtown Manhattan. But it might as well be a world away. In switching networks at a pivotal point in her career, Ms. Kelly, the No.   personality in cable television news, is taking a calculated risk that she can swap her dedicated Fox audience for the broader, but more fickle, viewership of network television. There are challenges from the  . Her splashy arrival has the potential to fray nerves among the big personalities at the network  ?   who already compete against one another for interviews and scoops. Ms. Kelly, 46, will also be taking on a daytime talk show format that has been a virtual graveyard for television news personalities in the last 10 years. And the new Sunday newsmagazine show that NBC plans to build around Ms. Kelly will go up against a giant that has not been meaningfully challenged for decades: ?60 Minutes? on CBS. Still, Ms. Kelly is a bona f

In [75]:
# https://www.geeksforgeeks.org/python-tokenize-text-using-textblob/
import spacy
from textblob import TextBlob

def sentence_split(text, split_type='spacy'):
	# Use the SpaCy model for sentence tokenization
	if split_type == 'spacy':
		# Load the SpaCy model
		nlp = spacy.load('en_core_web_sm')
		doc = nlp(text)
		sentences = [sent.text.strip() for sent in doc.sents]
		return sentences

	# Use the Textblob model for sentence tokenization
	elif split_type == 'textblob':
		blob_object = TextBlob(text)
		return [(str(i)) for i in blob_object.sentences]

	elif split_type == 'common':
		return [text.strip() for text in text.split('.') if text]

text =  """
Let?s just be very clear about this: There is no movie called ?Hidden Fences. ? But you would be forgiven for thinking otherwise after watching the Golden Globes, after the nonexistent movie was mentioned twice. The first time was definitely a mistake. The second time is not so clear. First, NBC?s Jenna Bush Hager, taking her first turn at Golden Globes reporting, presumably meant to ask Pharrell Williams about his best original score nomination for ?Hidden Figures,? a movie about three black women who would play crucial roles in the American space program. Surely she didn?t mean to ask him about ?Fences,? a similarly renowned film starring Viola Davis and Denzel Washington. Mr. Williams had no involvement in that. But: Mr. Williams?s   reaction, in which he appeared to be doing his best to hold back whatever he badly wished to say, was impressive in its restraint. Unsurprisingly, the immediate reaction on Twitter was less restrained. ?So Jenna Bush is just merging all of the black movies tonight at the #GoldenGlobes, huh?? one user asked. ?Pharrell?s face just held hundreds of years of whitewashing resolve,? another said. Others had fun in similarly inventing new movie titles. That seemed to be the end of things, but for some reason, Michael Keaton also referred to ?Hidden Fences? when listing the nominees for best supporting actress in a motion picture. Since the camera was focused on the actresses, it?s unclear if Mr. Keaton said it with a wink and a nod.
"""

context = sentence_split(text, 'common')
# context = coreference_resolution(context)
print(len(context))
context

18


['Let?s just be very clear about this: There is no movie called ?Hidden Fences',
 '? But you would be forgiven for thinking otherwise after watching the Golden Globes, after the nonexistent movie was mentioned twice',
 'The first time was definitely a mistake',
 'The second time is not so clear',
 'First, NBC?s Jenna Bush Hager, taking her first turn at Golden Globes reporting, presumably meant to ask Pharrell Williams about his best original score nomination for ?Hidden Figures,? a movie about three black women who would play crucial roles in the American space program',
 'Surely she didn?t mean to ask him about ?Fences,? a similarly renowned film starring Viola Davis and Denzel Washington',
 'Mr',
 'Williams had no involvement in that',
 'But: Mr',
 'Williams?s   reaction, in which he appeared to be doing his best to hold back whatever he badly wished to say, was impressive in its restraint',
 'Unsurprisingly, the immediate reaction on Twitter was less restrained',
 '?So Jenna Bush i

### Coreference Resolution utility

In [76]:
from fastcoref import spacy_component
import spacy

def coreference_resolution(context, model = 'fastcoref'):
	
	nlp = spacy.load('en_core_web_sm', exclude=["parser", "lemmatizer", "ner", "textcat"])
	# setting
	if model == 'fastcoref':
		nlp.add_pipe(
			"fastcoref", 
			config={
				"enable_progress_bar": False,
				}
			)
	elif model == 'LingMessCoref':
		nlp.add_pipe(
			"fastcoref", 
			config={
				"enable_progress_bar": False,
				'model_architecture': 'LingMessCoref', 'model_path': 'biu-nlp/lingmess-coref'
				}
			)
	else:
		print("please select model fastcoref or LingMessCoref ")
		
	doc = nlp(
	 	context, 
		component_cfg={"fastcoref": {'resolve_text': True}}
		)
	return doc._.resolved_text

coref_model = 'fastcoref'
test_results = coreference_resolution(test_sentence, model = coref_model)
print("Check the function coreference")
print(f"Before: {test_sentence}")
print(f"A Coref: {test_results}")
print(f"A Pre&Coref: {preprocess_text(test_results)}")

Map: 100%|██████████| 1/1 [00:00<00:00, 34.82 examples/s]


Check the function coreference
Before: Megyn Kelly?s new office at NBC News sits a block north of Fox News headquarters in Midtown Manhattan. But it might as well be a world away. In switching networks at a pivotal point in her career, Ms. Kelly, the No.   personality in cable television news, is taking a calculated risk that she can swap her dedicated Fox audience for the broader, but more fickle, viewership of network television. There are challenges from the  . Her splashy arrival has the potential to fray nerves among the big personalities at the network  ?   who already compete against one another for interviews and scoops. Ms. Kelly, 46, will also be taking on a daytime talk show format that has been a virtual graveyard for television news personalities in the last 10 years. And the new Sunday newsmagazine show that NBC plans to build around Ms. Kelly will go up against a giant that has not been meaningfully challenged for decades: ?60 Minutes? on CBS. Still, Ms. Kelly is a bona 

### Text matching utility

In [77]:
from sentence_transformers import SentenceTransformer, util

sematic_model = {
	'Mpnet_cos': SentenceTransformer('multi-qa-mpnet-base-cos-v1'), # 420 MB
	'distilbert_cos': SentenceTransformer('multi-qa-distilbert-cos-v1'), # 250 MB
	'MiniLM_cos': SentenceTransformer('multi-qa-MiniLM-L6-cos-v1'), # 80 MB
	'Mpnet_dot': SentenceTransformer('multi-qa-mpnet-base-dot-v1'), # 420 MB
	'distilbert_dot': SentenceTransformer('multi-qa-distilbert-dot-v1'), # 250 MB
	'MiniLM_dot': SentenceTransformer('multi-qa-MiniLM-L6-dot-v1'), # 80 MB
}

In [78]:
reranker_model = {
	'MiniLM_L6': SentenceTransformer('msmarco-MiniLM-L6-cos-v5'), # 22.7M 
	'distilbert': SentenceTransformer('msmarco-distilbert-cos-v5') # 66.4M
}

In [79]:
match_model = sematic_model['MiniLM_cos']

In [80]:
def semantic_similarity(sentence1, sentence2, model = sematic_model['Mpnet_cos']):
	embedding1 = model.encode(sentence1, convert_to_tensor=True)
	embedding2 = model.encode(sentence2, convert_to_tensor=True)
	cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
	return cosine_scores.item()

In [81]:
def retrieve_reranker(sentence1, sentence2, model = reranker_model['MiniLM_L6']):
	embedding1 = model.encode(sentence1, convert_to_tensor=True)
	embedding2 = model.encode(sentence2, convert_to_tensor=True)
	cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)
	return cosine_scores.item()

In [82]:
sematic_model.keys()

dict_keys(['Mpnet_cos', 'distilbert_cos', 'MiniLM_cos', 'Mpnet_dot', 'distilbert_dot', 'MiniLM_dot'])

In [83]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

def fine_tune_model(DataSet, model):
	
	train_examples = []
	for i in range(len(DataSet)):
		example = DataSet[i]
		for j in range(3):
			train_examples.append(InputExample(texts=[example['query'], example['pos'][0], example['neg'][j]]))
	
 	# Define your train dataset, the dataloader and the train loss
	train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)
	train_loss = losses.TripletLoss(model)
 
	# Tune the model
	model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=10)
	return model

DataSet = [
	{
		"query": "President Trump, asked by an interviewer on Saturday why he respected President Vladimir V.", 
		"pos": [ "president trump asked interviewer saturday president trump respected president vladimir v" ], 
		"neg": [
			"putin russia", 
			"putin russia even though president vladimir v", 
			"interview broadcast sunday super bowl president trump asked president trump respected president vladimir v"
   			]
		},
	{
		"query": "Although North Korea has vowed to develop the ability to attack the United States with nuclear warheads and has tested missiles that can cover the Korean Peninsula and its vicinity, it has never tested a missile that could fly over the Pacific", 
		"pos": [ "lthough north korea vowed develop ability attack united state nuclear warhead tested missile cover korean peninsula korean peninsula 's vicinity north korea never tested missile could fly pacific" ], 
		"neg": [
			"two leader shared need firm action persuade north korea rethink north korea 's nuclear weapon development program vowed respond strongly north korea attempted military provocation mr", 
			"new year day speech north korea leader said kim north korea leader 's country reached final stage preparing test intercontinental ballistic missile", 
			"next day president trump said twitter post happen remains unclear close north korea building reliable intercontinental ballistic missile although north korea boasted successfully testing crucial technology past year engine heat shield missile"
   			]
		},
	]

model = reranker_model['MiniLM_L6']

fine_tune_model(DataSet, model)

Epoch: 100%|██████████| 10/10 [00:01<00:00,  8.00it/s]


SentenceTransformer(
  (0): Transformer({'max_seq_length': 384, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [17]:
# Extract Answer
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the QA model and tokenizer
model_name_qa = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name_qa)
model_qa = AutoModelForQuestionAnswering.from_pretrained(model_name_qa)

def find_top_relevant_answer(question, predicted_sentence):
	
	inputs = tokenizer.encode_plus(question, predicted_sentence, return_tensors='pt', max_length=512, truncation_strategy='only_second')
	input_ids = inputs['input_ids']
	attention_mask = inputs['attention_mask']

	# Pass the encoded input through the QA model
	outputs = model_qa(input_ids, attention_mask=attention_mask, return_dict=True)
	start_logits = outputs.start_logits
	end_logits = outputs.end_logits


	# Decode the predicted start and end positions to get the answer
	start_index = torch.argmax(start_logits)
	end_index = torch.argmax(end_logits) + 1

	# Skip over any tokens before the start position or after the end position
	for j, token_id in enumerate(input_ids[0]):
		if j < start_index or j >= end_index:
			input_ids[0][j] = tokenizer.pad_token_id

	# Decode the answer from the corresponding tokens
	answer_tokens = input_ids[0][start_index:end_index]
	predicted_answer = tokenizer.decode(answer_tokens)
	
	return predicted_answer

In [56]:
# Extract Answer
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the QA model and tokenizer
model_name_qa = "distilbert-base-cased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name_qa)
model_qa = AutoModelForQuestionAnswering.from_pretrained(model_name_qa)

def find_top_relevant_answer(question, predicted_sentence):
	
	inputs = tokenizer.encode_plus(question, predicted_sentence, return_tensors='pt', max_length=512, truncation_strategy='only_second')
	input_ids = inputs['input_ids']
	attention_mask = inputs['attention_mask']

	# Pass the encoded input through the QA model
	outputs = model_qa(input_ids, attention_mask=attention_mask, return_dict=True)
	start_logits = outputs.start_logits
	end_logits = outputs.end_logits


	# Decode the predicted start and end positions to get the answer
	start_index = torch.argmax(start_logits)
	end_index = torch.argmax(end_logits) + 1

	# Skip over any tokens before the start position or after the end position
	for j, token_id in enumerate(input_ids[0]):
		if j < start_index or j >= end_index:
			input_ids[0][j] = tokenizer.pad_token_id

	# Decode the answer from the corresponding tokens
	answer_tokens = input_ids[0][start_index:end_index]
	predicted_answer = tokenizer.decode(answer_tokens)
	
	return predicted_answer

In [19]:
def find_top_relevant_sentences(question, article, match_model, top_n=1, min_confidence=0.5, c_stop = True, c_lower = True, c_lemma = True, coref_model = coref_model, split_type='spacy'):
	
	# Preprocess the question and article
	question_pre = preprocess_text(coreference_resolution(question, model = coref_model), c_stop = c_stop, c_lower = c_lower, c_lemma = c_lemma)
	article_pre = preprocess_text(coreference_resolution(article, model = coref_model), c_stop = c_stop, c_lower = c_lower, c_lemma = c_lemma)
	
	# Split the preprocessed article into sentences
	article_sentences = sentence_split(article_pre, split_type=split_type)
	
	# Initialize the list to hold top sentences with their similarity scores
	top_sentences_with_scores = []
	
	if not article_sentences:
		return [("No valid sentences found after processing.", 0.0,"")]
		
	if match_model == 'CosineTfidf':
		# Create the vectorizer and transform the sentences to get their feature vectors
		vectorizer = TfidfVectorizer()
		vectorizer.fit([question_pre] + article_sentences)
		question_vec = vectorizer.transform([question_pre])
		sentences_vec = vectorizer.transform(article_sentences)
		# Calculate cosine similarity scores between the question and each sentence
		similarity_scores = cosine_similarity(question_vec, sentences_vec).flatten()
  
		# Pair each sentence with its similarity score, filtering by minimum confidence
		for score, sentence in zip(similarity_scores, article_sentences):
			if score >= min_confidence:
				top_sentences_with_scores.append((
											sentence, 
											round(score, 3), 
											find_top_relevant_answer(question_pre, sentence)
											))
	
	else: 
		# Calculate similarity scores for each sentence
		for sent in article_sentences:
			similarity_score = retrieve_reranker(question_pre,sent, match_model)
			if similarity_score >= min_confidence:
				top_sentences_with_scores.append((
											sent, 
											round(similarity_score, 3), 
											find_top_relevant_answer(question_pre, sent)
											))
	
	# Sort the sentences by similarity score in descending order
	top_sentences_with_scores.sort(key=lambda x: x[1], reverse=True)
	
	# Take only the top_n sentences
	if top_sentences_with_scores == []:
		return [("no answer.", 0.0,"")]
	top_sentences_with_scores = top_sentences_with_scores[:top_n]
	
	return top_sentences_with_scores if top_sentences_with_scores else [("No sentences meet the minimum confidence score.", 0.0,"")]



In [38]:
(df[df.index == 18128])['article'].iloc[0]

'SEOUL, South Korea  ?   President Trump assured South Korea?s acting president on Monday of the United States? ?ironclad? commitment to defend the country, agreeing with Seoul to strengthen joint defense capabilities against North Korea. Mr. Trump?s assurances came amid anxiety in South Korea over the future of the alliance with the United States. During his campaign, Mr. Trump cast some doubt on the United States? defense and trade commitments, saying that South Korea was not paying enough to help keep 28, 500 American troops in the country. But speaking by phone to Hwang   the acting president of South Korea, Mr. Trump said that the coming visit to South Korea by Defense Secretary Jim Mattis reflected the close friendship of the two countries and the importance of their alliance. Mr. Mattis is scheduled to visit South Korea on Thursday on his first official trip abroad, which also includes a stop in Japan. ?President Trump reiterated our ironclad commitment to defend the R. O. K. in

### User Interface

#### Positive case

In [40]:
questions = "Who suggest that Ms. Kelly's performance at NBC will be as closely watched in the industry as her past few months of contract negotiations?"
id = 17340
top_sentences_with_scores = find_top_relevant_sentences(questions, (df[df.index == id])['article'].iloc[0], reranker_model['MiniLM_L6'], top_n=1)
predicted_sentence, confidence_score, predicted_answer = top_sentences_with_scores[0] if top_sentences_with_scores else ("No answer found", 0.0, " ")
print(f"User -- input from article {id}: {questions}")
print(f"System -- Answer with {confidence_score} confident score: {predicted_answer}")
print(f"System -- Related Sentence: {predicted_sentence}")


Map: 100%|██████████| 1/1 [00:00<00:00, 49.01 examples/s]


User -- input from article 17340: Who suggest that Ms. Kelly's performance at NBC will be as closely watched in the industry as her past few months of contract negotiations?
System -- Answer with 0.729 confident score: fox news nbc news rival channel
System -- Related Sentence: interview tuesday network executive producer fox news nbc news rival channel suggest megyn kelly performance nbc news closely watched industry megyn kelly 's past month contract negotiation .


#### Base cases

In [42]:
questions = 'Whom asked by an interviewer on Saturday why he respected President Vladimir V. Putin of Russia even though he is a killer?'
id = 17340
top_sentences_with_scores = find_top_relevant_sentences(questions, (df[df.index == id])['article'].iloc[0], reranker_model['MiniLM_L6'], top_n=1)
predicted_sentence, confidence_score, predicted_answer = top_sentences_with_scores[0] if top_sentences_with_scores else ("No answer found", 0.0, " ")
print(f"User -- input from article {id}: {questions}")
print(f"System -- Answer with {confidence_score} confident score: {predicted_answer}")
print(f"System -- Sentence: {predicted_sentence}")

Map: 100%|██████████| 1/1 [00:00<00:00, 37.49 examples/s]


User -- input from article 17340: Whom asked by an interviewer on Saturday why he respected President Vladimir V. Putin of Russia even though he is a killer?
System -- Answer with 0.0 confident score: 
System -- Sentence: no answer.


### Test utility

In [43]:
from rouge_score import rouge_scorer

def calculate_rouge_scores(reference, prediction):
	scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
	scores = scorer.score(reference.lower(), prediction.lower())
	return round(scores['rouge1'].fmeasure,3), round(scores['rougeL'].fmeasure,3)

ground_truth = "NBC and other rival channels  ?   suggest that Ms. Kelly?s performance at NBC will be as closely watched in the industry as her past few months of contract negotiations"
pred = "interview tuesday network executive producer fox news nbc news rival channel suggest megyn kelly performance nbc news closely watched industry megyn kelly 's past month contract negotiation"

print(f" ROUGE1 & L: {calculate_rouge_scores(ground_truth, pred)}")

04/13/2024 18:59:39 - INFO - 	 Using default tokenizer.


 ROUGE1 & L: (0.536, 0.5)


In [44]:
from rouge_score import rouge_scorer
from sklearn.metrics import f1_score

def calculate_individual_metrics(predicted_sentence, gold_standard):
	scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
	scores = scorer.score(gold_standard.lower(), predicted_sentence.lower())
	rouge1 = scores['rouge1'].fmeasure
	rougeL = scores['rougeL'].fmeasure
	
	# Assuming binary presence of each word for F1 calculation
	predicted = [1 if word in predicted_sentence.lower().split() else 0 for word in gold_standard.lower().split()]
	gold = [1 for _ in gold_standard.lower().split()]
	f1 = f1_score(gold, predicted, zero_division=1) if gold_standard else 0
	
	return f1, rouge1, rougeL

### Experimental Design

In [45]:
def evaluate_sentence_with_metrics(article_ids, questions, gold_standards, df, match_model = match_model, top_n=10, min_confidence=0.1, c_stop = True, c_lower = True, c_lemma = True, coref_model = coref_model, split_type='spacy', model_qa = model_qa):
	results = []

	for article_id, question, gold_standard in zip(article_ids, questions, gold_standards):
		processed_article = str(df.loc[article_id, 'article'])
		topic = str(df.loc[article_id, 'topic'])
		top_sentences = find_top_relevant_sentences(question, processed_article, match_model, top_n = top_n , min_confidence = min_confidence, c_stop = c_stop, c_lower = c_lower, c_lemma = c_lemma, coref_model = coref_model, split_type=split_type)
		predicted_sentence, confidence_score, predicted_answer = top_sentences[0] if top_sentences else ("No answer found", 0.0)

		f1, rouge1, rougeL = calculate_individual_metrics(predicted_answer, gold_standard)
		
		results.append({
			'Article ID': article_id,
			'Topic': topic,
			'Question': question,
			'Gold Standard': gold_standard,
			'Predicted Sentence': predicted_sentence,
			'Predicted Answer': predicted_answer,
			'Confidence Score': confidence_score,
			'F1-Score': f1,
			'ROUGE-1': rouge1,
			'ROUGE-L': rougeL,
		})

	results_df = pd.DataFrame(results)
	
	return results_df

In [156]:
def evaluate_articles_with_metrics(article_ids, questions, gold_standards, df, match_model = match_model, top_n=10, min_confidence=0.1, similarity_threshold=0.7, c_stop = True, c_lower = True, c_lemma = True, coref_model = coref_model, split_type='spacy'):
	evaluation_results = []
	reciprocal_ranks = []
	average_precisions = []

	for article_id, question, gold_standard in zip(article_ids, questions, gold_standards):
		processed_article = (df[df.index == article_id])['article'].iloc[0]
		topic = (df[df.index == article_id])['topic'].iloc[0]
		# top_sentences = find_top_relevant_sentences(question, processed_article, vectorizer, top_n=top_n, min_confidence=min_confidence)
		top_sentences = find_top_relevant_sentences(question, processed_article, match_model, top_n = top_n , min_confidence = min_confidence, c_stop = c_stop, c_lower = c_lower, c_lemma = c_lemma, coref_model = coref_model, split_type=split_type)
		
		top_matching_sentences = []
		first_relevant_rank = None
		relevant_hits = 0
		cumulative_precision = 0.0
		highest_similarity = 0
		highest_similarity_rank = None
		highest_similarity_sentence = ""

		for idx, (sentence, score, answer_) in enumerate(top_sentences, start=1):
			similarity = semantic_similarity(sentence, gold_standard)
			if similarity > highest_similarity:
				highest_similarity = similarity
				highest_similarity_rank = idx
				highest_similarity_sentence = sentence
			
			# using the similarity_threshold to check with the ground truth without maunally
			if similarity >= similarity_threshold:
				relevant_hits += 1
				if first_relevant_rank is None:
					first_relevant_rank = idx
				precision_at_this_rank = relevant_hits / idx
				cumulative_precision += precision_at_this_rank
			
			top_matching_sentences.append({
				'Sentence': sentence,
				'Rank': idx,
				'Score': round(score, 2),
				'Similarity': round(similarity, 2)
			})

		# Calculate AP for the current question
		ap = round(cumulative_precision / relevant_hits if relevant_hits > 0 else 0,2)
		rr = round(1 / first_relevant_rank if first_relevant_rank else 0,2)

		evaluation_results.append({
			'Article ID': article_id,
			'Topic': topic,
			'Question': question,
			'Gold Standard': gold_standard,
			'Reciprocal Rank': rr,
			'Average Precision': ap,
			'Top Matching Sentences': top_matching_sentences,
			'Highest Similarity Rank': highest_similarity_rank,
			'Highest Similarity Sentence': highest_similarity_sentence
		})

		reciprocal_ranks.append(rr)
		average_precisions.append(ap)

	# Convert evaluation results to DataFrame
	results_df = pd.DataFrame(evaluation_results)
	MRR = np.mean(reciprocal_ranks)
	MAP = np.mean(average_precisions)

	return results_df, MRR, MAP

### DEV - finetune parameter

In [143]:
reranker_model.keys()

dict_keys(['MiniLM_L6', 'distilbert'])

In [144]:
# Example usage with dataset
nlp_model = spacy.load('en_core_web_sm')  # Make sure to add your coreference resolution pipe if needed
vectorizer = TfidfVectorizer()

data = df_dev

# Dummy questions and gold standards for illustration; replace with your actual data
article_ids = (data.index.to_list())
questions = data['questions'].to_list()
gold_sentences = data['gold_sentences'].to_list()
gold_standards = data['gold answer'].to_list()

# Evaluate - setting parameter
# match_model = sematic_model['MiniLM_cos'] # or CosineTfidf
match_model = reranker_model['MiniLM_L6']
# match_model = 'CosineTfidf'
c_stop = False
c_lower = True
c_lemma = False
split_type = 'textblob' # spacy or textblob or common
coref_model = 'fastcoref' # fastcoref or LingMessCoref 

results_df_dev = evaluate_sentence_with_metrics(article_ids, questions, gold_standards, df, match_model = match_model, c_stop = c_stop, c_lower = c_lower, c_lemma = c_lemma, coref_model = coref_model, split_type=split_type)
article_df_dev, MRR_dev, MAP_dev = evaluate_articles_with_metrics(article_ids, questions, gold_sentences, df, match_model = match_model, c_stop = c_stop, c_lower = c_lower, c_lemma = c_lemma, coref_model = coref_model, split_type=split_type)

Map: 100%|██████████| 1/1 [00:00<00:00, 34.41 examples/s]


In [145]:
results_df_dev

,Article ID,Topic,Question,Gold Standard,Predicted Sentence,Predicted Answer,Confidence Score,F1-Score,ROUGE-1,ROUGE-L
0,17340,business,Where Megyn Kelly?s new office?,nbc news,megyn kelly s new office at nbc news sits a bl...,nbc news,0.828,1.000000,1.000000,1.000000
1,17623,business,What caused the shutdown of Laurel Canyon Boul...,Waterlogged porch collapse,the rain this week shut down laurel canyon bou...,the rain,0.713,0.000000,0.000000,0.000000
2,17623,business,How long did the drought in California last?,Six years,the arrival of another winter rain here a wet ...,six years,0.643,1.000000,1.000000,1.000000
3,17623,business,Whose concert marked a significant event in mu...,Johnny Cash's,the band actually performed twice in the morni...,the band actually performed twice in the morni...,0.480,0.000000,0.000000,0.000000
4,17559,business,Who responded to the claims in the opposition ...,Kellyanne Conway,one of memos generated by political operatives...,political operatives,0.670,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
95,17314,crime,What did the gunman who killed at least 39 pe...,assault,the turkish authorities are still searching fo...,a police officer guarding an istanbul nightclub,0.583,0.000000,0.000000,0.000000
96,17314,crime,Who is the government spokesman?,Numan Kurtulmus,the government s spokesman numan kurtulmus did...,numan kurtulmus,0.355,1.000000,1.000000,1.000000
97,17314,crime,What happened in November?,a deadly car bombing,a rare exception came in november when the isl...,car bombing,0.325,0.666667,0.666667,0.666667
98,17314,crime,How many The Turkish military said on Monday t...,at least 22 militants,the turkish military said on monday that the t...,at least 22,0.951,0.857143,0.857143,0.857143


In [146]:
# Evaluate
# Print Summary
print("Summary Article Metric Evaluation:")
print(f"MRR: {MRR_dev:.2f}")
print(f"MAP: {MAP_dev:.2f}")

print("Show the performance of sentence comparison")
print(" AVG F1 score: {:.2f}, AVG ROUGE-1: {:.2f}, AVG ROUGE-L: {:.2f}".format(
	np.mean(results_df_dev['F1-Score']),
	np.mean(results_df_dev['ROUGE-1']),
	np.mean(results_df_dev['ROUGE-L'])))
print("Show the performance of information retrieval")

Summary Article Metric Evaluation:
MRR: 0.69
MAP: 0.66
Show the performance of sentence comparison
 AVG F1 score: 0.54, AVG ROUGE-1: 0.48, AVG ROUGE-L: 0.48
Show the performance of information retrieval


In [147]:
sentence_index = 1
print(f"Question: {article_df_dev['Question'][sentence_index]}")
print(f"Gold Standard Benchmark: {article_df_dev['Gold Standard'][sentence_index]}")
check_results = pd.DataFrame(article_df_dev['Top Matching Sentences'][sentence_index])
check_results = check_results.reindex(['Rank','Score','Similarity','Sentence'], axis=1)
display_df(check_results)

Question: What caused the shutdown of Laurel Canyon Boulevard?
Gold Standard Benchmark: The waterlogged porch of a house collapsed into the roadway, causing the shutdown of Laurel Canyon Boulevard


,Rank,Score,Similarity,Sentence
0,1,0.71,0.80,the rain this week shut down laurel canyon boulevard after the waterlogged porch of a house collapsed into the roadway and flash flood warnings were issued in los angeles as storms swept back and forth across los angeles on thursday evening .
1,2,0.40,0.51,mudslides closed interstate 80 as the storm passed through the sierra nevada .
2,3,0.26,0.17,many years have started off wet and gone completely dry .
3,4,0.24,0.31,in northern california storms produced extensive flooding in sonoma county and around sacramento among the parts of california .
4,5,0.22,0.28,the arrival of another winter rain here a wet end to a week that began with heavy snows in the sierra nevada has begged a welcome question is the drought that punished california for six years over no question california has turned a corner .
5,6,0.22,0.08,assembled in the heavily guarded dining room 2 the inmates couldn t get enough .
6,7,0.22,0.27,reservoirs that were parched last year are close to capacity .
7,8,0.21,0.02,it was on this day in 1968 that johnny cash played johnny cash at folsom prison concert .
8,9,0.21,0.03,a year later johnny cash did another prison record at san quentin which was also successful .
9,10,0.20,0.22,los angeles and on thursday it rained .


#### System Evaluation on 10 test sets

In [150]:
# Example usage with dataset
nlp_model = spacy.load('en_core_web_sm')  # Make sure to add your coreference resolution pipe if needed
vectorizer = TfidfVectorizer()

# Dummy questions and gold standards for illustration; replace with your actual data
article_ids = [
	17340, 
	18303,
	17654,
	17726,
	17285,
	17999,
	17306,
	17312,
	17952,
	18128
	]
questions = [
	"Who suggest that Ms. Kelly's performance at NBC will be as closely watched in the industry as her past few months of contract negotiations?", 
	'Whom asked by an interviewer on Saturday why he respected President Vladimir V. Putin of Russia even though he is a killer?',
	"Who wrote the novel The Exorcist was both a milestone in horror fiction and a turning point in his own career?",
	"what is the topic the American Museum of Natural History has long been on the front lines for discussion?",
	"When Walt Disneys Bambi opened, which critics praised its spare, haunting visual style, vastly different from anything Disney had done before?",
	"When the group of scientists who orchestrate the Doomsday Clock moved its minute hand from three to two and a half minutes before the final hour?",
	"What Mariah Carey suffered through a performance train wreck in Times Square on New Year?s Eve?", 
	"What happened at an Istanbul nightclub on New Year's Day?",
	"Where the turbines would be connected to a substation?",
	"which country has vowed to develop the ability to attack the United States with nuclear warheads?"
	]
gold_sentences = [
	"Interviews on Tuesday with network executives and producers from Fox, NBC and other rival channels suggest that Ms. Kelly?s performance at NBC will be as closely watched in the industry as her past few months of contract negotiations.", 
	"President Trump, asked by an interviewer on Saturday why he respected President Vladimir V. Putin of Russia even though he is a killer, seemed to equate Mr. Putin?s actions with those of the United States.",
	'William Peter Blatty, the author whose book The Exorcist was both a milestone in horror fiction and a turning point in his own career.',
	"The American Museum of Natural History has long been on the front lines of the climate change discussion, as its scientists study the potential damage and its educators try to alert new generations to the dangers of global warming.",
	"When Walt Disney?s ?Bambi? opened in 1942, critics praised its spare, haunting visual style, vastly different from anything Disney had done before.",
	"On Thursday, the group of scientists who orchestrate the Doomsday Clock, a symbolic instrument informing the public when the earth is facing imminent disaster, moved its minute hand from three to two and a half minutes before the final hour.",
	"Mariah Carey suffered through a performance train wreck in Times Square on New Year?s Eve as malfunctions left her at a loss vocally during her hit song Emotions,struggling to reach notes and to sync the lyrics and music.",
	"The Turkish authorities are hunting for the gunman who opened fire at an Istanbul nightclub on New Year's Day, killing at least 39 people from no fewer than 12 countries.",
	"The turbines, each roughly 600 feet tall, would be connected to a substation in East Hampton by a undersea cable.",
	"North Korea has vowed to develop the ability to attack the United States with nuclear warheads and has tested missiles that can cover the Korean Peninsula and its vicinity, it has never tested a   missile that could fly over the Pacific."
	]
gold_standards = [
	"Fox, NBC and other rival channels", 
	"President Trump.",
	'William Peter Blatty',
	"climate change",
	"1942",
	"Thursday",
	"suffered through a performance train wreck",
	"gunman opened fire",
	"East Hampton by a undersea cable.",
	"North Korea"
	]

# Evaluate - setting parameter
match_model = sematic_model['distilbert_cos'] # or CosineTfidf
c_stop = False
c_lower = True
c_lemma = False
split_type = 'textblob' # spacy or textblob or common
coref_model = 'fastcoref' # fastcoref or LingMessCoref 

results_df_dev = evaluate_sentence_with_metrics(article_ids, questions, gold_standards, df, match_model = match_model, c_stop = c_stop, c_lower = c_lower, c_lemma = c_lemma, coref_model = coref_model, split_type=split_type)
article_df_dev, MRR, MAP = evaluate_articles_with_metrics(article_ids, questions, gold_sentences, df, match_model = match_model, c_stop = c_stop, c_lower = c_lower, c_lemma = c_lemma, coref_model = coref_model, split_type=split_type)
# Evaluate
# Print Summary
print("Summary Article Metric Evaluation:")
print(f"MRR: {MRR:.2f}")
print(f"MAP: {MAP:.2f}")

print("Show the performance of sentence comparison")
print(" AVG F1 score: {:.2f}, AVG ROUGE-1: {:.2f}, AVG ROUGE-L: {:.2f}".format(
	np.mean(results_df_dev['F1-Score']),
	np.mean(results_df_dev['ROUGE-1']),
	np.mean(results_df_dev['ROUGE-L'])))
print("Show the performance of information retrieval")

Map: 100%|██████████| 1/1 [00:00<00:00, 81.73 examples/s]


Summary Article Metric Evaluation:
MRR: 1.00
MAP: 0.96
Show the performance of sentence comparison
 AVG F1 score: 0.71, AVG ROUGE-1: 0.69, AVG ROUGE-L: 0.69
Show the performance of information retrieval


In [154]:
results_df_dev

,Article ID,Topic,Question,Gold Standard,Predicted Sentence,Predicted Answer,Confidence Score,F1-Score,ROUGE-1,ROUGE-L
0,17340,business,Who suggest that Ms. Kelly's performance at NB...,"Fox, NBC and other rival channels",interviews on tuesday with network executives ...,network executives and producers from fox news...,0.823,0.909091,0.631579,0.631579
1,18303,business,Whom asked by an interviewer on Saturday why h...,President Trump.,president trump asked by an interviewer on sat...,president trump,0.875,0.666667,1.000000,1.000000
2,17654,entertainment,Who wrote the novel The Exorcist was both a mi...,William Peter Blatty,william peter blatty the author whose book the...,william peter blatty,0.705,1.000000,1.000000,1.000000
3,17726,business,what is the topic the American Museum of Natur...,climate change,the american museum of natural history has lon...,climate change discussion,0.782,1.000000,0.800000,0.800000
4,17285,entertainment,"When Walt Disneys Bambi opened, which critics ...",1942,when walt disney s bambi opened in 1942 critic...,1942,0.920,1.000000,1.000000,1.000000
5,17999,entertainment,When the group of scientists who orchestrate t...,Thursday,on thursday the group of scientists who orches...,thursday,0.784,1.000000,1.000000,1.000000
6,17306,entertainment,What Mariah Carey suffered through a performan...,suffered through a performance train wreck,mariah carey suffered through a performance tr...,,0.851,0.000000,0.000000,0.000000
7,17312,politics,What happened at an Istanbul nightclub on New ...,gunman opened fire,here s what you need to know the turkish autho...,,0.702,0.000000,0.000000,0.000000
8,17952,politics,Where the turbines would be connected to a sub...,East Hampton by a undersea cable.,the turbines each roughly 600 feet tall would ...,east hampton,0.732,0.500000,0.500000,0.500000
9,18128,politics,which country has vowed to develop the ability...,North Korea,although north korea has vowed to develop the ...,north korea,0.582,1.000000,1.000000,1.000000


In [157]:
results_df[['Gold Standard','Predicted Sentence']]
for i in range(len(results_df)):
	print(f"Query {i} semantic similarity score: {round(semantic_similarity(results_df['Gold Standard'][i], results_df['Predicted Answer'][i]),2)}")
	print(f"Question: " + results_df['Question'][i])
	print(f"Gold Sentence: " + gold_sentences[i])
	print(f"Related Sentence: " + results_df['Predicted Sentence'][i])
	print(f"Gold Standard: " + results_df['Gold Standard'][i])
	print(f"Predicted Sentence: " + results_df['Predicted Answer'][i])
	print("---------------------------")

Query 0 semantic similarity score: 0.67
Question: Who suggest that Ms. Kelly's performance at NBC will be as closely watched in the industry as her past few months of contract negotiations?
Gold Sentence: Interviews on Tuesday with network executives and producers from Fox, NBC and other rival channels suggest that Ms. Kelly?s performance at NBC will be as closely watched in the industry as her past few months of contract negotiations.
Related Sentence: interviews on tuesday with network executives and producers from fox news nbc news and other rival channels suggest that megyn kelly s performance at nbc news will be as closely watched in the industry as megyn kelly s 's past few months of contract negotiations .
Gold Standard: Fox, NBC and other rival channels
Predicted Sentence: network executives and producers from fox news nbc news and other rival channels
---------------------------
Query 1 semantic similarity score: 0.81
Question: Whom asked by an interviewer on Saturday why he r

In [155]:
sentence_index = 7
print(f"Question: {article_df_dev['Question'][sentence_index]}")
print(f"Gold Standard Benchmark: {article_df_dev['Gold Standard'][sentence_index]}")
check_results = pd.DataFrame(article_df_dev['Top Matching Sentences'][sentence_index])
check_results = check_results.reindex(['Rank','Score','Similarity','Sentence'], axis=1)
display_df(check_results)

Question: What happened at an Istanbul nightclub on New Year's Day?
Gold Standard Benchmark: The Turkish authorities are hunting for the gunman who opened fire at an Istanbul nightclub on New Year's Day, killing at least 39 people from no fewer than 12 countries.


,Rank,Score,Similarity,Sentence
0,1,0.70,0.97,here s what you need to know the turkish authorities are hunting for the gunman who opened fire at an istanbul nightclub on new year s day killing at least 39 people from no fewer than 12 countries .
1,2,0.66,0.86,the islamic state claimed the gunman who opened fire at an istanbul nightclub on new year s day killing at least 39 people from no fewer than 12 countries as a hero soldier of the caliphate and appeared to refer to turkey s role in the syrian war .
2,3,0.30,0.38,the new york times suicide bombers struck the international airport in mogadishu somalia killing at least three security officers .
3,4,0.24,0.09,good morning .
4,5,0.24,0.18,fans around the world plan to toast the professor at 9 p. m. local time .
5,6,0.24,0.28,most major markets reopen after new year s holiday .
6,7,0.22,0.40,in iraq the islamic state claimed a suicide bombing in central baghdad that killed dozens even as the islamic state makes a brutal effort to hang on to the islamic state 's only remaining iraqi stronghold mosul .
7,8,0.21,0.13,tyrus wong who endured racial bias to become one of the most celebrated artists of the 20th century and whose influence was crucial to the animated film bambi died at 106. finally our asia correspondents don t limit our asia correspondents to traditional news stories .
8,9,0.20,0.34,al jazeera a prison battle in brazil between gangs fighting for control of the cocaine trade left about 60 inmates dead some decapitated .
9,10,0.18,0.13,enjoy .


## B. References

[1] Preprocessing - NLTK: https://www.nltk.org/

[2] Preprocessing - spaCy: https://spacy.io/usage/spacy-101

[3] Sentence splitting: https://towardsdatascience.com/building-a-question-answering-system-part-1-9388aadff507

[4] Model distilbert: https://huggingface.co/distilbert/distilbert-base-cased

[5] Transfermer: https://huggingface.co/transformers/v3.0.2/model_doc/auto.html

[6] Pretrained model + traineig method: https://www.sbert.net/docs/pretrained_models.html

[7] Fine-tuned model: https://huggingface.co/blog/how-to-train-sentence-transformers

** all the related theory provided the reference on report

## C. Appendix